In [ ]:
import os # Operating System
import numpy as np
import pandas as pd
import datetime as dt # Datetime
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
import geocoder
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes
import folium #import folium # map rendering library

print('Libraries imported.')

In [ ]:
# Set the present working directory
os.chdir("/resources/data/Coursera")

# Read the data for examination
df_ppd = pd.read_csv("Price_Paid_Data.csv")

# Assign meaningful column names
df_ppd.columns = ['TUID', 'Price', 'Date_Transfer', 'Postcode', 'Prop_Type', 'Old_New', 'Duration', 'PAON', \
                  'SAON', 'Street', 'Locality', 'Town_City', 'District', 'County', 'PPD_Cat_Type', 'Record_Status']

In [ ]:
# Format the date column
df_ppd['Date_Transfer'] = df_ppd['Date_Transfer'].apply(pd.to_datetime)

# Sort by Date of Sale
df_ppd.sort_values(by=['Date_Transfer'],ascending=[False],inplace=True)

# Add a column with YEAR
df_ppd['Year'] = df_ppd['Date_Transfer'].dt.year

# Drop all transactions before 2016
df_ppd.drop(df_ppd[df_ppd.Date_Transfer.dt.year < 2016].index, inplace=True)

In [ ]:
# Shape of my dataset
df_ppd.shape

In [ ]:
# Let's find the shape again
df_ppd.shape

In [ ]:
df_grp_year = df_ppd.groupby(['Year']).size()
df_grp_year

In [ ]:
df_ppd_london = df_ppd.query("Town_City == 'LONDON'")

In [ ]:
districts = df_ppd_london['District'].unique().tolist()
len(districts)

In [ ]:
town_city = df_ppd_london['Town_City'].unique().tolist()
len(town_city)

In [ ]:
streets = df_ppd_london['Street'].unique().tolist()
len(streets)

In [ ]:
london_data = pd.read_csv("Street Coordinates London.csv")
london_data['Street'] = london_data['Street'].str.replace(', LONDON', '')

In [ ]:
london_data.head(5)

In [ ]:
df_grp_postcode.sort_values(ascending=[False],inplace=True)
df_grp_postcode.head(10)

In [ ]:
address = 'London, UK'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London City are {}, {}.'.format(latitude, longitude))

In [ ]:
# create map of Manhattan using latitude and longitude values
map_london = folium.Map(location=[latitude, longitude], zoom_start=15)

# add markers to map
for lat, lng, label in zip(london_data['Latitude'], london_data['Longitude'], london_data['Street']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
map_london

In [ ]:
df_grp_price = df_ppd_london.groupby(['Street'])['Price'].mean().reset_index()
df_grp_price.columns = ['Street', 'Avg_Price']

In [ ]:
df_affordable = df_grp_price.query("(Avg_Price >= 2000000) & (Avg_Price <= 2200000)")

In [ ]:
df_affordable

In [ ]:
df_myhouse = pd.merge(london_data, df_affordable, on=['Street'], how='inner')

In [ ]:
df_myhouse

In [ ]:
address = 'London, UK'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London City are {}, {}.'.format(latitude, longitude))

In [ ]:
# create map of Manhattan using latitude and longitude values
map_london = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, price, street in zip(df_myhouse['Latitude'], df_myhouse['Longitude'], df_myhouse['Avg_Price'], df_myhouse['Street']):
    label = '{}, {}'.format(street, price)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='black',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
map_london

